# Learning Classifiers from Scratch

## Model Source

The initial, basic learning classifier system model taken from Dr. Ryan Urbanowicz's "Learning Classifier Systems in a Nutshell" video on YouTube found here: https://youtu.be/CRge_cZ2cJc?si=1CM2osKW7CptJ-DM

This video description of an LCS is the simplest and most digestable that has been found while also staying complete in terms of LCS operation. Additionally, some psuedo code snippets have been taken from Dr. Martin Butz's book "Rule-Based Evolutionary Online Learning Systems" and his algorithmic description of XCS.

### Step 1: Initialize Setup

Initialize the population and create the functions for creating empty match sets and action sets:

In [1326]:
# Initialize the empty population. This is only called once at the beginning of the cycle.
def initialize_population():
    population = []
    return population

population = initialize_population()
print(population)

[]


In [1327]:
print(population)

[]


### Step 2: Feeding Data to LCS

LCS is an online learning mechanism, but will normally be trained from some dataset. Data from the dataset in training or from the environment in testing will need to be fed to the LCS.

In [1328]:

data = './6Multiplexer_Data_Complete.csv'

# Get the length of the file so that the get_instance function doesn't return anything if requested line is not present
def get_data_length(data):
    with open(data, 'r') as file:
        return sum(1 for row in file)

def convert_int(instance):
    int_instance = []
    for i in instance:
        int_instance.append(int(i))
    return int_instance

# Create a function that gets the data from a file an returns a specified instance of the dataset to the LCS
def get_instance(data, line_num):
    import csv
    lines = get_data_length(data)
    with open(data, 'r') as source:
        reader = csv.reader(source)
        if line_num > lines:
            return
        for _ in range(line_num):
            next(reader)
        return convert_int(next(reader))

instance = get_instance(data, 1)
print(instance)

[0, 0, 0, 0, 0, 0, 0]


### Step 3: Determine if classifiers in population match the current instance

Compare each classifier in the population to the current instance. If classifiers in the population match, they are each added to the match set.

In [1329]:
# Create a does_match function that compares each attribute between two classifiers
def does_match(state, instance):
    for i in range(len(state)):
        index = state[i][0]
        if state[i][1] != instance[index]:
            return False
    return True

# Create the match set by comparing the attributes of each classifier in the population with the current instance
def create_match_set(population, instance):
    match_set = []
    if len(population) == 0:
        return match_set
    else:
        for classifier in population:
            state = classifier['state']
            if does_match(state, instance) == True:
                match_set.append(classifier)
                classifier['match count'] += 1
                classifier['accuracy'] = classifier['correct count'] / classifier['match count']
                classifier['fitness'] = classifier['accuracy'] ** 5
        return match_set

match_set = create_match_set(population, instance)
print(population)
print(match_set)

[]
[]


### Step 4: Generate the correct set

From the match set, create a correct set by comparing the action or class of each classifier with the action or class of each instance.

In [1330]:
# Create the correct set by comparing the class or action of each classifier in the match set with the current instance

def create_correct_set(match_set, instance):
    correct_set = []
    if len(match_set) == 0:
        return correct_set
    else:
        for classifier in match_set:
            if classifier['action'] == instance[-1]:
                correct_set.append(classifier)
                classifier['correct count'] +=1
                classifier['accuracy'] = classifier['correct count'] / classifier['match count']
                classifier['fitness'] = classifier['accuracy'] ** 5
        return correct_set
        
correct_set = create_correct_set(match_set, instance)
print(population)
print(correct_set)

[]
[]


### Step 5: Covering

In most LCS, the population is initialized as being empty. Covering adds classifiers to the population using the current instance if the correct set is empty. This is also the step that turns the simple instance data into the classifier dictionary.

In [1331]:
# Create a dictionary item to represent the current instance if the correct set is empty.

def covering(instance, iteration, specificity):
    import random
    state = []
    action = instance[-1]
    for x in range(len(instance) - 1):
        if random.random() < specificity:
            state.append(tuple((x, instance[x])))
        classifier = {'state': state, 
                        'action': action, 
                        'numerosity': 1, 
                        'match count': 1, 
                        'correct count': 1, 
                        'accuracy': 1, 
                        'fitness': 1, 
                        'birth iteration': iteration}
    return classifier

classifier = covering(instance, 1, specificity=.5)

def update_population(classifier, population):
    population.append(classifier)
update_population(classifier, population)
print(population)

[{'state': [(0, 0)], 'action': 0, 'numerosity': 1, 'match count': 1, 'correct count': 1, 'accuracy': 1, 'fitness': 1, 'birth iteration': 1}]


In [1332]:
def testing(data):
    population = initialize_population()
    length = get_data_length(data)
    for i in range(1, length):
        instance = get_instance(data, i)
        match_set = create_match_set(population, instance)
        correct_set = create_correct_set(match_set, instance)
        if len(correct_set) == 0:
            classifier = covering(instance, i, specificity=.5)
            update_population(classifier, population)
    return population

In [1333]:
testing(data)

[{'state': [(4, 0), (5, 0)],
  'action': 0,
  'numerosity': 1,
  'match count': 16,
  'correct count': 12,
  'accuracy': 0.75,
  'fitness': 0.2373046875,
  'birth iteration': 1},
 {'state': [(1, 0), (4, 0)],
  'action': 0,
  'numerosity': 1,
  'match count': 15,
  'correct count': 11,
  'accuracy': 0.7333333333333333,
  'fitness': 0.21208362139917689,
  'birth iteration': 2},
 {'state': [(0, 0), (1, 0), (2, 0), (5, 0)],
  'action': 0,
  'numerosity': 1,
  'match count': 3,
  'correct count': 3,
  'accuracy': 1.0,
  'fitness': 1.0,
  'birth iteration': 3},
 {'state': [(2, 0), (3, 0), (5, 1)],
  'action': 0,
  'numerosity': 1,
  'match count': 7,
  'correct count': 4,
  'accuracy': 0.5714285714285714,
  'fitness': 0.06092699470458736,
  'birth iteration': 4},
 {'state': [(1, 0), (2, 0), (3, 1), (5, 1)],
  'action': 0,
  'numerosity': 1,
  'match count': 3,
  'correct count': 2,
  'accuracy': 0.6666666666666666,
  'fitness': 0.13168724279835387,
  'birth iteration': 8},
 {'state': [(0, 0)